<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2022notebooks/2022_1020lam_making_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%config InlineBackend.figure_format = 'retina'
try:
    import bit
except ImportError:
    !pip install ipynbname --upgrade > /dev/null 2>&1 
    !git clone https://github.com/ShinAsakawa/bit.git
import bit

isColab = bit.isColab
HOME = bit.HOME

if isColab:
    # 近藤先生のご指摘によりインストールされる全ライブラリのメッセージを表示させることにした
    !apt install aptitude
    !aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
    !pip install mecab-python3==0.7
    !pip install -q konoha[all]
    !pip install jaconv
    !pip install japanize_matplotlib
    
    import MeCab
    wakati = MeCab.Tagger('-Owakati').parse
    yomi = MeCab.Tagger('-Oyomi').parse
else:
    from ccap.mecab_settings import yomi
    from ccap.mecab_settings import wakati


In [ ]:
# 自作ライブラリ LAM の読み込み
if isColab:
    !git clone https://github.com/ShinAsakawa/ccap.git
    !git clone https://github.com/ShinAsakawa/lam.git

In [ ]:
# # upload `psylex71utf8.txt` from local drive
# # Be carefull! This file is copyright protected. 
# from google.colab import files
# uploaded = files.upload()
# !mv psylex71utf8.txt ccap

In [ ]:
# ここはお遊びなので，スキップしても良い
import IPython
IPython.display.Image(url="https://livedoor.blogimg.jp/ftb001/imgs/b/4/b4629a79.jpg")

In [ ]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn
import torch.optim

# 自作ライブラリ LAM の読み込み
import lam 
from lam import EncoderRNN
from lam import AttnDecoderRNN
from lam import convert_ids2tensor
from lam import train
from lam import asMinutes, timeSince
#from lam import fit
from lam import convert_ids2tensor
from lam import fix_seed
from lam import worker_init_fn
from lam import make_vocab_dataset

import os
import sys
import typing
import numpy as np
import random
import pandas as pd
import gzip
import copy
from termcolor import colored

# from tqdm import tqdm         #commandline で実行時
from tqdm.notebook import tqdm  #jupyter で実行時

# from tqdm import tqdm         #commandline で実行時
from tqdm.notebook import tqdm  #jupyter で実行時

# シミュレーションに必要なパラメータの設定
params = {
    'traindata_size':   10000,    # 訓練データ数，NTT 日本語語彙特性の高頻度語を上位から抽出
    #'traindata_size': 301612,    # 訓練データ数，NTT 日本語語彙特性の高頻度語を上位から抽出
    'epochs': 20,                # 学習のためのエポック数
    'hidden_size': 24,           # 中間層のニューロン数
    'random_seed': 42,           # 乱数の種。ダグラス・アダムス著「銀河ヒッチハイカーズガイド」

    # 以下 `source` と `target` を定義することで，別の課題を実行可能
    'source': 'orthography',        # ['orthography', 'phonology', 'mora', 'mora_p', 'mora_p_r']
    'target': 'mora_p_r',          # ['orthography', 'phonology', 'mora', 'mora_p', 'mora_p_r']
    # 'orthography': 書記素, 
    # 'phonology': 音韻, 
    # 'mora': モーラ
    # 'mora_p': モーラを silius による音分解
    # 'mora_p_r': モーラの silius 音分解の逆
    'pretrained': False,          # True であれば訓練済ファイルを読み込む
    #'pretrained': True,          # True であれば訓練済ファイルを読み込む
    'isTrain'   : True,          # True であれば学習する
    
    # 学習済のモデルパラメータを保存するファイル名
    #'path_saved': '2022_0607lam_o2p_hid32_vocab10k.pt', 
    #'path_saved': '2022_0829lam_p2p_hid24_vocab10k.pt',
    'path_saved': False,                      # 保存しない場合
    
    # 結果の散布図を保存するファイル名    
    'path_graph': '2022_0829lam_p2p_hid24_vocab10k.pdf',
    #'path_graph': False,                      # 保存しない場合

    'lr': 0.001,                              # 学習率
    'dropout_p': 0.0,                         # ドロップアウト率
    'teacher_forcing_ratio': 0.5,             # 教師強制を行う確率
    'optim_func': torch.optim.Adam,           # 最適化アルゴリズム ['torch.optim.Adam', 'torch.optim.SGD', 'torch.optim.AdamW']
    'loss_func' :torch.nn.CrossEntropyLoss(), # 交差エントロピー損失 ['torch.nn.NLLLoss()', or 'torch.nn.CrossEntropyLoss()']
}

device = lam.device  # CPU or GPU の選択

for param in params:
    print(colored(f'{param}','blue',attrs=['bold']), colored(f': {params[param]}','grey'))
    
# 乱数シード固定（再現性担保のため）
fix_seed(seed = params['random_seed'])
print(worker_init_fn(1))    

_vocab = lam.VOCAB(traindata_size=params['traindata_size'], yomi=yomi)

In [ ]:
# 結果を保存するなら，このセルを実行
import json
tmp_data = {}
for x in dir(_vocab):
    if not x.startswith('__'):
        if isinstance(eval(f'_vocab.{x}'), dict):
            if len(eval(f'_vocab.{x}')) != 0:
                tmp_data[x] = eval(f'_vocab.{x}')
                print(colored((x, len(tmp_data[x])), 'blue', attrs=['bold']))
        elif isinstance(eval(f'_vocab.{x}'), list):
            tmp_data[x] = eval(f'_vocab.{x}')
            print(colored((x, len(tmp_data[x])), 'red', attrs=['bold']))
        elif isinstance(eval(f'_vocab.{x}'), int):
            tmp_data[x] = eval(f'_vocab.{x}')
            print(colored((x, tmp_data[x]), 'green', attrs=['bold']))
        elif isinstance(eval(f'_vocab.{x}'), str):
            tmp_data[x] = eval(f'_vocab.{x}')
            print(colored((x, tmp_data[x]), 'yellow', attrs=['bold']))
        else:
            print(x, type(eval(f'_vocab.{x}')))

#zip_fname = '2022_1011lam_traindata.json.gz'
#zip_fname = '2022_1011lam_traindata10k.json.gz'
zip_fname = '2022_1018lam_traindata10k.json.gz'
with gzip.open(zip_fname, 'wt', encoding='utf-8') as fp:
    json.dump(tmp_data,fp)
        
with gzip.open(zip_fname, 'r') as fp:
    x = json.load(fp)

print(tmp_data.keys())
print(x.keys())